In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
spark = SparkSession.builder.appName('stroke').getOrCreate()
from pyspark.ml.feature import (VectorAssembler,OneHotEncoder,
                               StringIndexer)
from pyspark.ml import Pipeline

In [2]:
#Creating the spark session
#spark = SparkSession.builder.master("local[2]").appName("myApp").getOrCreate()

In [3]:
#Spark session info- connection
spark

In [4]:
#Spark context enviroment accessed
#sc = spark.sparkContext

In [5]:
#Reading the raw data without the schema
# f = sc.textFile("train_2v.csv")
f= spark.read.csv("train_2v.csv", inferSchema=True, header =True)

In [6]:
f

DataFrame[id: int, gender: string, age: double, hypertension: int, heart_disease: int, ever_married: string, work_type: string, Residence_type: string, avg_glucose_level: double, bmi: double, smoking_status: string, stroke: int]

In [7]:
#Fetch list from RDD(Spark term form resiliant distrubuted dataset)
f.collect()

[Row(id=30669, gender='Male', age=3.0, hypertension=0, heart_disease=0, ever_married='No', work_type='children', Residence_type='Rural', avg_glucose_level=95.12, bmi=18.0, smoking_status=None, stroke=0),
 Row(id=30468, gender='Male', age=58.0, hypertension=1, heart_disease=0, ever_married='Yes', work_type='Private', Residence_type='Urban', avg_glucose_level=87.96, bmi=39.2, smoking_status='never smoked', stroke=0),
 Row(id=16523, gender='Female', age=8.0, hypertension=0, heart_disease=0, ever_married='No', work_type='Private', Residence_type='Urban', avg_glucose_level=110.89, bmi=17.6, smoking_status=None, stroke=0),
 Row(id=56543, gender='Female', age=70.0, hypertension=0, heart_disease=0, ever_married='Yes', work_type='Private', Residence_type='Rural', avg_glucose_level=69.04, bmi=35.9, smoking_status='formerly smoked', stroke=0),
 Row(id=46136, gender='Male', age=14.0, hypertension=0, heart_disease=0, ever_married='No', work_type='Never_worked', Residence_type='Rural', avg_glucose_l

In [8]:
# Define the schema, corresponding to a line in the csv data file.
schema = StructType([
    StructField("id", IntegerType(), nullable=True),
    StructField("gender", StringType(), nullable=True),
    StructField("age", IntegerType(), nullable=True),
    StructField("hypertension", IntegerType(), nullable=True),
    StructField("heart_disease", IntegerType(), nullable=True),
    StructField("ever_married", StringType(), nullable=True),
    StructField("Residence_type", StringType(), nullable=True),
    StructField("avg_glucose_level",IntegerType(), nullable=True),
    StructField("bmi", IntegerType(), nullable=True),
    StructField("smoking_status", StringType(), nullable=True),
    StructField("stroke", IntegerType(), nullable=True)
   ]
)

In [9]:
updated_f=f.na.fill('No Info', subset=['smoking_status'])


In [10]:
updated_f.take(5)


[Row(id=30669, gender='Male', age=3.0, hypertension=0, heart_disease=0, ever_married='No', work_type='children', Residence_type='Rural', avg_glucose_level=95.12, bmi=18.0, smoking_status='No Info', stroke=0),
 Row(id=30468, gender='Male', age=58.0, hypertension=1, heart_disease=0, ever_married='Yes', work_type='Private', Residence_type='Urban', avg_glucose_level=87.96, bmi=39.2, smoking_status='never smoked', stroke=0),
 Row(id=16523, gender='Female', age=8.0, hypertension=0, heart_disease=0, ever_married='No', work_type='Private', Residence_type='Urban', avg_glucose_level=110.89, bmi=17.6, smoking_status='No Info', stroke=0),
 Row(id=56543, gender='Female', age=70.0, hypertension=0, heart_disease=0, ever_married='Yes', work_type='Private', Residence_type='Rural', avg_glucose_level=69.04, bmi=35.9, smoking_status='formerly smoked', stroke=0),
 Row(id=46136, gender='Male', age=14.0, hypertension=0, heart_disease=0, ever_married='No', work_type='Never_worked', Residence_type='Rural', avg

In [11]:
# from pyspark.ml.feature import OneHotEncoder, StringIndexer

#One-Hot-Encoding for fields - gender, residence type and smoke_status
stringIndexer = StringIndexer(inputCol="gender", outputCol="genderIndex")
model = stringIndexer.fit(updated_f)
indexed = model.transform(updated_f)
encoder = OneHotEncoder(inputCol="genderIndex", outputCol="genderVec")
encoded = encoder.transform(indexed)
#encoded.show()

In [12]:
stringIndexer = StringIndexer(inputCol="Residence_type", outputCol="Residence_typeIndex")
model = stringIndexer.fit(updated_f)
indexed = model.transform(updated_f)
encoder = OneHotEncoder(inputCol="Residence_typeIndex", outputCol="Residence_typeVec")
encoded = encoder.transform(indexed)
#encoded.show()

In [13]:
stringIndexer = StringIndexer(inputCol="smoking_status", outputCol="smoking_Index")
model = stringIndexer.fit(updated_f)
indexed = model.transform(updated_f)
encoder = OneHotEncoder(inputCol="smoking_Index", outputCol="smoking_Vec")
encoded = encoder.transform(indexed)
encoded.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+-------------+-------------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|smoking_Index|  smoking_Vec|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+-------------+-------------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|        No Info|     0|          1.0|(3,[1],[1.0])|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|     0|          0.0|(3,[0],[1.0])|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|        No Info|     0|          1.0|(3,[1],[1.0])|
|56543|Female|70.0|   

In [14]:
stringIndexer = StringIndexer(inputCol="ever_married", outputCol="ever_marriedIndex")
model = stringIndexer.fit(updated_f)
indexed = model.transform(updated_f)
encoder = OneHotEncoder(inputCol="ever_marriedIndex", outputCol="ever_marriedVec")
encoded = encoder.transform(indexed)
#encoded.show()

In [15]:
assembler = VectorAssembler(inputCols=['genderVec',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedVec',
 'work_typeVec',
 'Residence_typeVec',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVec'],outputCol='features')

pipeline = Pipeline(stages=assembler)
updated_f = pipeline.fit(updated_f).transform(updated_f)
## View the assembler vector

TypeError: 'VectorAssembler' object is not iterable

In [ ]:
# Sanity check, operations and alayzing the data and visualization